In [1]:
from scratch.linear_algebra import Vector, dot

def step_function(x: float) -> float: 
    return 1.0 if x >= 0 else 0.0 

def perceptron_output(weights: Vector, bias: float, x: Vector) -> float: 
    """Returns 1 if the perceptron 'fires', 0 if not""" 
    calculation = dot(weights, x) + bias 
    return step_function(calculation)

In [2]:
and_weights = [2., 2] 
and_bias = -3. 
 
assert perceptron_output(and_weights, and_bias, [1, 1]) == 1 
assert perceptron_output(and_weights, and_bias, [0, 1]) == 0 
assert perceptron_output(and_weights, and_bias, [1, 0]) == 0 
assert perceptron_output(and_weights, and_bias, [0, 0]) == 0

In [3]:
or_weights = [2., 2] 
or_bias = -1. 
 
assert perceptron_output(or_weights, or_bias, [1, 1]) == 1 
assert perceptron_output(or_weights, or_bias, [0, 1]) == 1 
assert perceptron_output(or_weights, or_bias, [1, 0]) == 1 
assert perceptron_output(or_weights, or_bias, [0, 0]) == 0


In [4]:
not_weights = [-2.] 
not_bias = 1. 
 
assert perceptron_output(not_weights, not_bias, [0]) == 1 
assert perceptron_output(not_weights, not_bias, [1]) == 0

In [5]:
and_gate = min 
or_gate = max 
xor_gate = lambda x, y: 0 if x == y else 1

In [6]:
import math

def sigmoid(x: float) -> float: 
    return 1.0 / (1.0 + math.exp(-x))

In [7]:
def neuron_output(weights: Vector, inputs: Vector) -> float: 
    # weights includes the bias term, inputs includes a 1 
    return sigmoid(dot(weights, inputs))

In [10]:
from typing import List
from scratch.linear_algebra import Vector

def feed_forward(neural_network: List[Vector], input_vector: Vector) -> List[float]: 
    """ 
    Feeds the input vector through the neural network. 
    Returns the outputs of all layers (not just the last one). 
    """ 
    outputs: List[Vector] = [] 
 
    for layer in neural_network: 
        input_with_bias = input_vector + [1]              # Add a constant. 
        output = [neuron_output(neuron, input_with_bias)  # Compute the output 
                  for neuron in layer]                    # for each neuron. 
        outputs.append(output)                            # Add to results. 
 
        # Then the input to the next layer is the output of this one 
        input_vector = output 
 
    return outputs

In [11]:
xor_network = [# hidden layer 
               [[20., 20, -30],      # 'and' neuron 
                [20., 20, -10]],     # 'or'  neuron 
               # output layer 
               [[-60., 60, -30]]]

# feed_forward returns the outputs of all layers, so the [-1] gets the 
# final output, and the [0] gets the value out of the resulting vector 
assert 0.000 < feed_forward(xor_network, [0, 0])[-1][0] < 0.001 
assert 0.999 < feed_forward(xor_network, [1, 0])[-1][0] < 1.000 
assert 0.999 < feed_forward(xor_network, [0, 1])[-1][0] < 1.000 
assert 0.000 < feed_forward(xor_network, [1, 1])[-1][0] < 0.001